In [ ]:
#@markdown ## 구글드라이브 연동
#@markdown **생성한 이미지를 구글드라이브에 자동저장하려면 체크해주세요**
Google_Drive = False #@param {type:"boolean"}

#@markdown **초기모델(checkpoint)을 구글드라이브에 저장하려면 체크해주세요**
#@markdown <div><font color="red">모델 파일사이즈(2GB ~ 7GB)가 크니 구글드라이브 용량을 확인해주세요</div>
#@markdown <div><font color="red">대신 한번받으면 다음 실행부터는 빨라져요</div>
Checkpoint_Google_Save = False #@param {type:"boolean"}

#@markdown -----
#@markdown *터널링*
#@markdown **Ngrok**
Ngrok_Key = '' #@param {type:"string"}

#@markdown **Localtunnel**
Localtunnel = False #@param {type:"boolean"}

#@markdown -----
#@markdown *아래는 설정할 필요 없어요*
Google_Drive_Dir = 'webui' #@param {type:"string"}

# 모델/리소스 기본값
Checkpoint_Url = 'https://civitai.com/api/download/models/56071?type=Model&format=SafeTensor&size=pruned&fp=fp16' #@param {type:"string"}
Checkpoint_Filename = 'dark_sushi_mix.safetensors' #@param {type:"string"}

# 확장 사용 여부
ControlNet = True #@param {type:"boolean"}
ControlNet_Extra = True #@param {type:"boolean"}
ControlNet_IPAdapter = False #@param {type:"boolean"}
ControlNet_T2I = False #@param {type:"boolean"}
DDetailer = True #@param {type:"boolean"}
ADetailer = False #@param {type:"boolean"}
SegmentAnything = False #@param {type:"boolean"}

# 워크스페이스/브랜치
Workspace = 'ui'
NotebookVersion = 'stable'

# ========= 기본 준비 =========
import os, json, shutil, textwrap, subprocess, sys, pathlib

if Google_Drive:
    from google.colab import drive
    drive.mount('/content/drive')

# 경로
ROOT = '/content'
WEBUI_DIR = f'{ROOT}/{Workspace}'
MODELS_SD = f'{WEBUI_DIR}/models/Stable-diffusion'
MODELS_VAE = f'{WEBUI_DIR}/models/VAE'
MODELS_ESRGAN = f'{WEBUI_DIR}/models/ESRGAN'
MODELS_LORA = f'{WEBUI_DIR}/models/Lora'
EMBED_DIR = f'{WEBUI_DIR}/embeddings'
EXT_DIR = f'{WEBUI_DIR}/extensions'
CN_MODELS = f'{EXT_DIR}/controlnet/models'

# 체크포인트 저장 경로(드라이브에 저장 옵션일 때 별도 폴더)
Checkpoint_SavePath = MODELS_SD
if Google_Drive and Checkpoint_Google_Save:
    Checkpoint_SavePath = f'{MODELS_SD}/google'

# 패키지
!apt -y install -qq aria2 > /dev/null

# ========= WebUI 설치(토치 호환 버전으로) =========
%cd /content
if not os.path.exists(WEBUI_DIR):
    !git clone --depth 1 https://github.com/AUTOMATIC1111/stable-diffusion-webui {Workspace}

# 최신 pip & 토치/토치비전/cu121 + xformers (CUDA 12.1 기준)
!python -m pip -q install --upgrade pip
!pip -q install --extra-index-url https://download.pytorch.org/whl/cu121 \
  torch==2.5.1+cu121 torchvision==0.20.1+cu121 torchaudio==2.5.1+cu121
# xformers는 속도용(미설치도 동작). 2.5 계열 호환 버전:
!pip -q install xformers==0.0.28.post2 triton

# WebUI 나머지 의존성
%cd {WEBUI_DIR}
# requirements에 박힌 구버전 토치 충돌 방지: torch/torchvision 라인은 무시하고 설치
!pip -q install -r requirements_versions.txt --no-deps
!pip -q install -r requirements.txt --no-deps

# ========= 확장 설치 =========
%cd {EXT_DIR}
# ControlNet
if ControlNet and not os.path.exists(f'{EXT_DIR}/sd-webui-controlnet'):
    !git clone --depth 1 https://github.com/Mikubill/sd-webui-controlnet
# ddetailer/adetailer/segment-anything
if DDetailer and not os.path.exists(f'{EXT_DIR}/ddetailer'):
    !git clone --depth 1 https://github.com/dustysys/ddetailer
if ADetailer and not os.path.exists(f'{EXT_DIR}/adetailer'):
    !git clone --depth 1 https://github.com/Bing-su/adetailer
if SegmentAnything and not os.path.exists(f'{EXT_DIR}/sd-webui-segment-anything'):
    !git clone --depth 1 https://github.com/continue-revolution/sd-webui-segment-anything

# ========= 구글드라이브 연동(선택) =========
if Google_Drive:
    DRIVE_BASE = f'/content/drive/MyDrive/{Google_Drive_Dir}/{NotebookVersion}'
    os.makedirs(DRIVE_BASE, exist_ok=True)
    # 폴더 구조 안내: 아래 폴더가 있으면 내용을 WebUI에 심볼릭 링크
    # webui/output, checkpoint, lora, embedding, hyperwork, wildcards …
    link_map = {
        f'{DRIVE_BASE}/output': f'{WEBUI_DIR}/outputs',
        f'{DRIVE_BASE}/checkpoint': MODELS_SD,
        f'{DRIVE_BASE}/lora': MODELS_LORA,
        f'{DRIVE_BASE}/embedding': EMBED_DIR,
        f'{DRIVE_BASE}/wildcards': f'{WEBUI_DIR}/wildcards',
        f'{DRIVE_BASE}/hyperwork': f'{WEBUI_DIR}/hyperwork'
    }
    for src, dst in link_map.items():
        os.makedirs(src, exist_ok=True)
        if os.path.islink(dst) or os.path.exists(dst):
            pass
        else:
            os.makedirs(os.path.dirname(dst), exist_ok=True)
            !ln -s "{src}" "{dst}"

# ========= 모델/리소스 다운로드 =========
# 디렉토리 생성
for d in [MODELS_SD, MODELS_VAE, MODELS_ESRGAN, MODELS_LORA, EMBED_DIR]:
    os.makedirs(d, exist_ok=True)

# upscaler
%cd {WEBUI_DIR}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth \
  -d {MODELS_ESRGAN} -o 4x-UltraSharp.pth

# checkpoint
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{Checkpoint_Url}" \
  -d {Checkpoint_SavePath} -o {Checkpoint_Filename}

# vae
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt \
  -d {Checkpoint_SavePath} -o vae-ft-mse-840000-ema-pruned.vae.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt \
  -d {MODELS_VAE} -o kl-f8-anime2.ckpt

# lora
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  "https://civitai.com/api/download/models/62833?type=Model&format=SafeTensor" \
  -d {MODELS_LORA} -o add_detail.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  "https://civitai.com/api/download/models/63006?type=Model&format=SafeTensor" \
  -d {MODELS_LORA} -o LowRA.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  "https://civitai.com/api/download/models/32988?type=Model&format=SafeTensor&size=full&fp=fp16" \
  -d {MODELS_LORA} -o blindbox_V1Mix.safetensors

# embeddings
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.pt \
  -d {EMBED_DIR} -o EasyNegative.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  https://huggingface.co/AsciiP/badhandv4/resolve/main/badhandv4.pt \
  -d {EMBED_DIR} -o badhandv4.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  https://huggingface.co/yesyeahvh/bad-hands-5/resolve/main/bad-hands-5.pt \
  -d {EMBED_DIR} -o bad-hands-5.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  "https://civitai.com/api/download/models/60938?type=Negative&format=Other" \
  -d {EMBED_DIR} -o negative_hand.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  "https://civitai.com/api/download/models/60095?type=Negative&format=Other" \
  -d {EMBED_DIR} -o bad_prompt_version2.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  "https://civitai.com/api/download/models/5637?type=Model&format=PickleTensor&size=full&fp=fp16" \
  -d {EMBED_DIR} -o ng_deepnegative_v1_75t.pt

# ControlNet 모델들
if ControlNet:
    os.makedirs(CN_MODELS, exist_ok=True)
    base = "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main"
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {base}/control_v11p_sd15_openpose_fp16.safetensors -d {CN_MODELS} -o control_v11p_sd15_openpose_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {base}/control_v11p_sd15_lineart_fp16.safetensors -d {CN_MODELS} -o control_v11p_sd15_lineart_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {base}/control_v11p_sd15_softedge_fp16.safetensors -d {CN_MODELS} -o control_v11p_sd15_softedge_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {base}/control_v11f1p_sd15_depth_fp16.safetensors -d {CN_MODELS} -o control_v11f1p_sd15_depth_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {base}/control_v11p_sd15_canny_fp16.safetensors -d {CN_MODELS} -o control_v11p_sd15_canny_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {base}/control_v11f1e_sd15_tile_fp16.safetensors -d {CN_MODELS} -o control_v11f1e_sd15_tile_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {base}/control_v11e_sd15_ip2p_fp16.safetensors -d {CN_MODELS} -o control_v11e_sd15_ip2p_fp16.safetensors
    if ControlNet_Extra:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {base}/control_v11p_sd15_seg_fp16.safetensors -d {CN_MODELS} -o control_v11p_sd15_seg_fp16.safetensors
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {base}/control_v11p_sd15s2_lineart_anime_fp16.safetensors -d {CN_MODELS} -o control_v11p_sd15s2_lineart_anime_fp16.safetensors
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {base}/control_v11p_sd15_inpaint_fp16.safetensors -d {CN_MODELS} -o control_v11p_sd15_inpaint_fp16.safetensors
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {base}/control_v11p_sd15_scribble_fp16.safetensors -d {CN_MODELS} -o control_v11p_sd15_scribble_fp16.safetensors
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {base}/control_v11p_sd15_mlsd_fp16.safetensors -d {CN_MODELS} -o control_v11p_sd15_mlsd_fp16.safetensors
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {base}/control_v11p_sd15_normalbae_fp16.safetensors -d {CN_MODELS} -o control_v11p_sd15_normalbae_fp16.safetensors
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {base}/control_v11e_sd15_shuffle_fp16.safetensors -d {CN_MODELS} -o control_v11e_sd15_shuffle_fp16.safetensors
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/CiaraRowles/TemporalNet/resolve/main/diff_control_sd15_temporalnet_fp16.safetensors -d {CN_MODELS} -o diff_control_sd15_temporalnet_fp16.safetensors
    if ControlNet_IPAdapter:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.bin -d {CN_MODELS} -o ip-adapter_sd15.pth
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.bin -d {CN_MODELS} -o ip-adapter-plus_sd15.pth
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.bin -d {CN_MODELS} -o ip-adapter-plus-face_sd15.pth
    if ControlNet_T2I:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -d {CN_MODELS} -o t2iadapter_style_sd14v1.pth
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -d {CN_MODELS} -o t2iadapter_color_sd14v1.pth
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd15v2.pth -d {CN_MODELS} -o t2iadapter_sketch_sd15v2.pth
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_zoedepth_sd15v1.pth -d {CN_MODELS} -o t2iadapter_zoedepth_sd15v1.pth
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ioclab_sd15_recolor.safetensors -d {CN_MODELS} -o ioclab_sd15_recolor.safetensors

# ========= config.json 기본 체크포인트 지정 =========
CFG = f'{WEBUI_DIR}/config.json'
if os.path.exists(CFG):
    !sed -i -e "s/\"sd_model_checkpoint\": \"\"/\"sd_model_checkpoint\": \"{Checkpoint_Filename}\"/g" {CFG}

# ========= 실행 =========
%cd {WEBUI_DIR}
common_flags = "--no-half-vae --theme dark --gradio-queue --skip-version-check --skip-torch-cuda-test"
if Ngrok_Key:
    !python launch.py --xformers --ngrok {Ngrok_Key} {common_flags}
else:
    # 로컬터널 옵션
    if Localtunnel:
        !npm install -g localtunnel
    !python launch.py --share --xformers {common_flags}
